In [15]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds
import inspect

In [16]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde.csv'

df_trans_werte = pd.read_csv(os.path.join(path, filename))

In [17]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde.csv'

df_LY = pd.read_csv(os.path.join(path, filename))

In [18]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

In [19]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_NO_Privat_ohneApple_Langzeitkunde.csv'

df_NO_werte = pd.read_csv(os.path.join(path, filename))

In [20]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

In [21]:
## Alle DataFrame Werte von Float zu int
df_trans_werte = df_trans_werte.astype(int)
df_LY = df_LY.astype(int)
df_HY = df_HY.astype(int)
df_NO_werte = df_NO_werte.astype(int)
df_LO = df_LO.astype(int)

# Transwerte-1 definiert für spätere Berecnungen in der Copula
#df_trans_werte_minusEins = df_trans_werte.loc[:, df_NO_werte.columns != 'iid'] -1
#df_trans_werte_minusEins = df_trans_werte_minusEins.astype(int)

In [22]:
len(df_trans_werte)

1462

In [23]:
len(df_LY)

1462

In [24]:
len(df_HY)

697

In [25]:
len(df_NO_werte)

647

In [26]:
len(df_LO)

647

### Modellframework Anwendung

In [27]:
# Definition der Modellparamater
# Anzahl states
anzahl_states = 3
# Parameter für Transitionmatrxi
alpha = np.zeros((anzahl_states, anzahl_states)).flatten()
gamma = np.zeros((anzahl_states, anzahl_states, 4)).flatten()
# Parameter für CEOM
alpha_o = [0] * anzahl_states
beta_o = [0]
#Parameter für CPM
delta_0 = [0] * anzahl_states
delta_1 = [0] * anzahl_states
alpha_p = [0] * anzahl_states
beta_p_1 = [0] * anzahl_states
beta_p_2 = [0] * anzahl_states
r = [0]
#Parameter für Frank_copula
theta = [0]

In [28]:
df_HY.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,2,2,2,3,2,3,2,2,2,1,1,2,2,3,1
1,100057957,2,2,2,3,2,3,2,2,2,1,1,2,2,3,1
2,100059553,2,2,2,3,2,3,2,2,2,1,1,2,2,3,1
3,100062565,2,2,2,3,2,3,2,2,2,1,1,2,2,3,1
4,100062713,2,2,2,3,2,3,2,2,2,1,1,2,2,3,1


In [29]:
df_trans_werte.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,100057957,1,1,0,1,0,1,0,1,0,0,0,1,0,1,1
2,100059553,2,0,0,0,0,1,0,0,1,0,0,0,0,0,0
3,100062565,4,0,1,3,2,1,1,2,1,1,0,1,2,3,0
4,100062713,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [30]:
df_LO.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1,1,1,1,1,2,3,1,2,1,2,1,2,3,1
1,100057957,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1
2,100059553,1,2,3,1,1,1,1,2,3,1,2,3,1,1,1
3,100062565,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,100062713,1,1,1,2,3,4,5,6,7,8,9,10,11,12,13


In [31]:
df_NO_werte.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1,1,1,3,0,0,1,0,1,0,1,0,0,1,0
1,100057957,2,2,2,3,2,3,2,0,2,1,1,2,2,3,0
2,100059553,0,0,1,1,1,1,0,0,1,0,0,1,1,1,0
3,100062565,2,2,2,3,2,3,2,2,1,1,1,2,2,3,1
4,100062713,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
df_LY.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1,1,2,3,4,5,6,7,8,9,10,11,12,13,1
1,100057957,1,1,1,2,1,2,1,2,1,2,3,4,1,2,1
2,100059553,1,1,2,3,4,5,1,2,3,1,2,3,4,5,6
3,100062565,1,1,2,1,1,1,1,1,1,1,1,2,1,1,1
4,100062713,1,1,2,3,4,5,6,7,1,2,3,4,5,6,7


In [33]:
# Entfernen der Zeilen aus df_HY, die nicht in df_trans_werte und df_LO enthalten sind
df_HY = df_HY[df_HY['iid'].isin(df_trans_werte['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LO['iid'])]

# Entfernen der Zeilen aus df_trans_werte, die nicht in df_HY, df_NO_werte und df_LY enthalten sind
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_HY['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_NO_werte['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_LY['iid'])]

# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LO = df_LO[df_LO['iid'].isin(df_HY['iid'])]

# Entfernen der Zeilen aus df_NO_werte, die nicht in df_trans_werte enthalten sind
df_NO_werte = df_NO_werte[df_NO_werte['iid'].isin(df_trans_werte['iid'])]

# Entfernen der Zeilen aus df_LY, die nicht in df_trans_werte enthalten sind
df_LY = df_LY[df_LY['iid'].isin(df_trans_werte['iid'])]

### Alle DataFrames Indizes resetten, damit man sie gemeinsam identifiziere kann
df_HY = df_HY.reset_index(drop=True)
df_trans_werte = df_trans_werte.reset_index(drop=True)
df_LO = df_LO.reset_index(drop=True)
df_NO_werte = df_NO_werte.reset_index(drop=True)
df_LY = df_LY.reset_index(drop=True)

# NO-1 definiert für spätere Berechnung inder Copula
df_NO_minusEins = df_NO_werte.copy()
df_NO_minusEins.loc[:, df_NO_minusEins.columns != 'iid'] = df_NO_minusEins.loc[:, df_NO_minusEins.columns != 'iid'] -1
df_NO_minusEins = df_NO_minusEins.astype(int)
# transwerte -1 definiert für spätere Berechnung inder Copula
df_trans_werte_minusEins = df_trans_werte.copy()
df_trans_werte_minusEins.loc[:, df_trans_werte_minusEins.columns != 'iid'] = df_trans_werte_minusEins.loc[:, df_trans_werte_minusEins.columns != 'iid'] -1
df_trans_werte_minusEins = df_trans_werte_minusEins.astype(int)

##### Datentest --- IGNORIEREN

In [34]:
df_NO_minusEins.info

<bound method DataFrame.info of            iid  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0    100056041  0  0  0  2 -1 -1  0 -1  0  -1   0  -1  -1   0  -1
1    100057957  1  1  1  2  1  2  1 -1  1   0   0   1   1   2  -1
2    100059553 -1 -1  0  0  0  0 -1 -1  0  -1  -1   0   0   0  -1
3    100062565  1  1  1  2  1  2  1  1  0   0   0   1   1   2   0
4    100062713  0  0 -1 -1 -1 -1 -1 -1 -1  -1  -1  -1  -1  -1  -1
..         ... .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..
642  158089087 -1  0  2  2  1  2  1  1  1   0   0   1   1   2   0
643  158089902 -1  1  0  2  1  2  1  1  1   0   0   1   1   1   0
644  158091516 -1  0 -1  0 -1 -1 -1 -1 -1  -1  -1  -1  -1  -1  -1
645  158158683 -1  0 -1  0  0 -1  0  0 -1  -1  -1   1  -1  -1   0
646  160436094 -1 -1 -1 -1 -1  1  1  1  1   0   0   1   1   1   0

[647 rows x 16 columns]>

In [35]:
df_HY.info

<bound method DataFrame.info of            iid  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0    100056041  2  2  2  3  2  3  2  2  2   1   1   2   2   3   1
1    100057957  2  2  2  3  2  3  2  2  2   1   1   2   2   3   1
2    100059553  2  2  2  3  2  3  2  2  2   1   1   2   2   3   1
3    100062565  2  2  2  3  2  3  2  2  2   1   1   2   2   3   1
4    100062713  2  2  2  3  2  3  2  2  2   1   1   2   2   3   1
..         ... .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..
642  158089087  0  2  2  3  2  3  2  2  2   1   1   2   2   3   1
643  158089902  0  2  2  3  2  3  2  2  2   1   1   2   2   3   1
644  158091516  0  2  2  3  2  3  2  2  2   1   1   2   2   3   1
645  158158683  0  2  2  3  2  3  2  2  2   1   1   2   2   3   1
646  160436094  0  0  0  0  0  2  2  2  2   1   1   2   2   2   1

[647 rows x 16 columns]>

In [36]:
df_trans_werte.info

<bound method DataFrame.info of            iid  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0    100056041  1  0  0  0  0  0  0  0  0   0   0   0   0   1   0
1    100057957  1  1  0  1  0  1  0  1  0   0   0   1   0   1   1
2    100059553  2  0  0  0  0  1  0  0  1   0   0   0   0   0   0
3    100062565  4  0  1  3  2  1  1  2  1   1   0   1   2   3   0
4    100062713  2  0  0  0  0  0  0  1  0   0   0   0   0   0   0
..         ... .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..
642  158089087  1  0  0  0  0  0  0  0  0   0   0   0   1   0   0
643  158089902  1  0  0  0  0  0  0  1  0   0   0   0   0   0   0
644  158091516  1  1  0  0  0  0  0  0  0   0   0   1   0   2   1
645  158158683  1  0  0  0  0  0  0  0  0   0   0   1   0   0   0
646  160436094  1  0  2  0  1  4  0  2  1   0   0   0   1   0   0

[647 rows x 16 columns]>

In [37]:
df_LO.info

<bound method DataFrame.info of            iid  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0    100056041  1  1  1  1  1  2  3  1  2   1   2   1   2   3   1
1    100057957  1  1  1  1  1  1  1  1  2   1   1   1   1   1   1
2    100059553  1  2  3  1  1  1  1  2  3   1   2   3   1   1   1
3    100062565  1  1  1  1  1  1  1  1  1   1   1   1   1   1   1
4    100062713  1  1  1  2  3  4  5  6  7   8   9  10  11  12  13
..         ... .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..
642  158089087  1  2  1  1  1  1  1  1  1   1   1   1   1   1   1
643  158089902  1  2  1  1  1  1  1  1  1   1   1   1   1   1   1
644  158091516  1  2  1  2  1  2  3  4  5   6   7   8   9  10  11
645  158158683  1  2  1  2  1  1  2  1  1   2   3   4   1   2   3
646  160436094  1  2  3  4  5  6  1  1  1   1   1   1   1   1   1

[647 rows x 16 columns]>

In [38]:
df_NO_werte.info

<bound method DataFrame.info of            iid  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0    100056041  1  1  1  3  0  0  1  0  1   0   1   0   0   1   0
1    100057957  2  2  2  3  2  3  2  0  2   1   1   2   2   3   0
2    100059553  0  0  1  1  1  1  0  0  1   0   0   1   1   1   0
3    100062565  2  2  2  3  2  3  2  2  1   1   1   2   2   3   1
4    100062713  1  1  0  0  0  0  0  0  0   0   0   0   0   0   0
..         ... .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..
642  158089087  0  1  3  3  2  3  2  2  2   1   1   2   2   3   1
643  158089902  0  2  1  3  2  3  2  2  2   1   1   2   2   2   1
644  158091516  0  1  0  1  0  0  0  0  0   0   0   0   0   0   0
645  158158683  0  1  0  1  1  0  1  1  0   0   0   2   0   0   1
646  160436094  0  0  0  0  0  2  2  2  2   1   1   2   2   2   1

[647 rows x 16 columns]>

In [39]:
df_LY.info

<bound method DataFrame.info of            iid  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0    100056041  1  1  2  3  4  5  6  7  8   9  10  11  12  13   1
1    100057957  1  1  1  2  1  2  1  2  1   2   3   4   1   2   1
2    100059553  1  1  2  3  4  5  1  2  3   1   2   3   4   5   6
3    100062565  1  1  2  1  1  1  1  1  1   1   1   2   1   1   1
4    100062713  1  1  2  3  4  5  6  7  1   2   3   4   5   6   7
..         ... .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..
642  158089087  1  1  2  3  4  5  6  7  8   9  10  11  12   1   2
643  158089902  1  1  2  3  4  5  6  7  1   2   3   4   5   6   7
644  158091516  1  1  1  2  3  4  5  6  7   8   9  10   1   2   1
645  158158683  1  1  2  3  4  5  6  7  8   9  10  11   1   2   3
646  160436094  1  1  2  1  2  1  1  2  1   1   2   3   4   1   2

[647 rows x 16 columns]>

In [40]:
df_trans_werte_minusEins.info

<bound method DataFrame.info of            iid  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0    100056041  0 -1 -1 -1 -1 -1 -1 -1 -1  -1  -1  -1  -1   0  -1
1    100057957  0  0 -1  0 -1  0 -1  0 -1  -1  -1   0  -1   0   0
2    100059553  1 -1 -1 -1 -1  0 -1 -1  0  -1  -1  -1  -1  -1  -1
3    100062565  3 -1  0  2  1  0  0  1  0   0  -1   0   1   2  -1
4    100062713  1 -1 -1 -1 -1 -1 -1  0 -1  -1  -1  -1  -1  -1  -1
..         ... .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..  ..
642  158089087  0 -1 -1 -1 -1 -1 -1 -1 -1  -1  -1  -1   0  -1  -1
643  158089902  0 -1 -1 -1 -1 -1 -1  0 -1  -1  -1  -1  -1  -1  -1
644  158091516  0  0 -1 -1 -1 -1 -1 -1 -1  -1  -1   0  -1   1   0
645  158158683  0 -1 -1 -1 -1 -1 -1 -1 -1  -1  -1   0  -1  -1  -1
646  160436094  0 -1  1 -1  0  3 -1  1  0  -1  -1  -1   0  -1  -1

[647 rows x 16 columns]>

##### Definition der Transitionmatrix

In [22]:
# Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state, monat):
    
    state_t_minusEins = state_t_minusEins -1
    state = state -1
    str_monat = str(monat)
    
    # Definition der X_t_minusEins
    # verwendete Parameter
    EM_monat_minusEins = df_HY[str_monat]
    Y_monat_minusEins = df_trans_werte[str_monat]
    O_monat_minusEins = df_NO_werte[str_monat]
    # Definition
    X_t_minusEins = [O_monat_minusEins.apply(lambda x: int(x > 0)),\
                     Y_monat_minusEins.apply(lambda x: int(x > 0)),\
                     EM_monat_minusEins, np.square(EM_monat_minusEins)]
    
    X_t_minusEins = np.array(X_t_minusEins)
    array_v = alpha[3*state_t_minusEins + state] +\
                    gamma[3*state_t_minusEins + state] * X_t_minusEins[0] +\
                    gamma[(np.square(anzahl_states)) + 3*state_t_minusEins + state] * X_t_minusEins[1] +\
                    gamma[(2* np.square(anzahl_states)) + 3*state_t_minusEins + state] * X_t_minusEins[2] +\
                    gamma[(3* np.square(anzahl_states)) + 3*state_t_minusEins + state] * X_t_minusEins[3]

    return array_v

# Definition der Transitionmatrix
def probability_Q_state_monat(alpha, gamma):
    
    ### Erstellung leerer anzahl_states x anzahl_states arrays für jeden Kunden und jeden Monat
    Q = []
    for iid in df_trans_werte["iid"]:
        for monat in range(2,16):
                empty_array = np.zeros((anzahl_states, anzahl_states))
                Q.append(empty_array)
                
    for monat in range(2,16):
        
        for state_t_minusEins in range(1, anzahl_states +1):
            
            # Erste Spalte besteht aus Nullen, da eine Trasition zurück in state 1 nicht möglich ist
            for state in range(2, anzahl_states +1):
                
                q_array_iid_monat_state = Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state, monat)

                # Speicherung der Werte in den leeren arrays
                for iid in range(len(q_array_iid_monat_state)):
                    
                    #Neue Definition ohne die Verwedung von Namen
                    Q[iid * (len(range(2,16))) + monat - 2][state_t_minusEins-1, state-1] = q_array_iid_monat_state[iid]
    # Wenn einmal in state 2, kann nicht mehr zurück in state 1
    for arr in Q:
        arr[0, 0] = 0.0
        arr[1, 0] = 0.0
        arr[2, 0] = 0.0
        
    for j in range(len(Q)):
        denominator = 1 + np.sum(np.exp(Q[j][:, 1:]), axis=1, keepdims=True)
        Q[j][:,1:] = np.exp(Q[j][:,1:]) / denominator
    

    return Q

#### Tests

##### Definition der Verteilungsfunktion F_1 von CEOM aals anzahl_kunden x 1 Array für jeden state und Monat

In [23]:
# Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def lambda_O_state_monat(alpha_o, beta_o, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
    EM_monat = df_HY[str_monat]
    LO_monat = df_LO[str_monat]
    
    # Erhalten der Werte für die Verarbeitung in der Formel
    array_lambda = np.exp(0.5*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))
    return array_lambda

# Array der Verteilungsfunktion der Öffungen O für Monat und state
def F_O_state_monat(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_werte[str_monat]
    
    lambda_O_state_monat_berechnung = lambda_O_state_monat(alpha_o, beta_o, state, monat)
    
    array_F_O_berechnung = poisson.cdf(o_monat, lambda_O_state_monat_berechnung)
         
    return array_F_O_berechnung

# Array der Verteilungsfunktion der Öffungen O -1 für Monat und state
def F_O_state_monat_minusEins(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_minusEins[str_monat]
    
    lambda_O_state_monat_berechnung = lambda_O_state_monat(alpha_o, beta_o, state, monat)
    
    array_F_O_berechnung = poisson.cdf(o_monat, lambda_O_state_monat_berechnung)

    return array_F_O_berechnung

#### Tests

##### Definition der Verteilungsfunktio F_2 von CPM als anzahl_kunden x 1 Array für jeden state und Monat

In [24]:
# Definition der Wahrscheinlichkeit des Eintretens eines Erfolges im Einzelversuch phi als anzahl_iids x T df
def phi_state_monat(delta_0, delta_1, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    LY_monat = df_LY[str_monat]
    array_phi = 1 / (1 + np.exp(delta_0[state] + delta_1[state] * np.log(LY_monat)))
    
    return array_phi

# Definition der lambdaY als anzahl_iids x 1 array für monat und state
def lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    EM_monat = df_HY[str_monat]
    
    array_lambdaY = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
    
    return array_lambdaY

# Array der Verteilungsfunktion der Käufe Y für Monat und state
def F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    # y_monat ist das array mit der Anzahl an Käufen in monat str_monat
    y_monat = df_trans_werte[str_monat]
    # phi ist das 1 x anzahl_iids array mit phi für jeden kunden mit gegegebenem monat und state
    phi = phi_state_monat(delta_0, delta_1, state, monat)
    # lambdaY ist das 1 x anzahl_iids array mit lammbda für jeden kunden mit gegegebenem monat und state
    lambdaY = lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat)
    # pmf als 1x anzahl_iids array mit Nullen initieren
    cdf = np.zeros_like(y_monat, dtype=float)
    
    if r[0] > 0:
        for i in range(len(y_monat)):
            for j in range(y_monat[i] + 1):
                if j == 0:
                    pmf = phi[i] + (1 - phi[i]) * (1 + lambdaY[i] / r[0])**(-r[0])
                elif j > 0:
                    numerator = math.gamma(j + r[0])
                    denominator = math.factorial(j) * math.gamma(r[0])
                    gamma_term = numerator / denominator
                    pmf= (1 - phi[i]) * gamma_term * (1 + lambdaY[i] / r[0])**(-r[0]) * (1 + r[0] / lambdaY[i])**(-j)
                cdf[i] += pmf
    return cdf

# Array der Verteilungsfunktion der Käufe Y-1 für Monat und state
def F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    # y_monat ist das array mit der Anzahl an Käufen in monat str_monat
    y_monat = df_trans_werte[str_monat] -1
    # phi ist das 1 x anzahl_iids array mit phi für jeden kunden mit gegegebenem monat und state
    phi = phi_state_monat(delta_0, delta_1, state, monat)
    # lambdaY ist das 1 x anzahl_iids array mit lammbda für jeden kunden mit gegegebenem monat und state
    lambdaY = lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat)
    # pmf als 1x anzahl_iids array mit Nullen initieren
    cdf = np.zeros_like(y_monat, dtype=float)
    
    if r[0] > 0:
        for i in range(len(y_monat)):
            if y_monat[i] < 0:
                cdf[i] = 0
            else:
                for j in range(y_monat[i] + 1):
                    if j == 0:
                        pmf = phi[i] + (1 - phi[i]) * (1 + lambdaY[i] / r[0])**(-r[0])
                    elif j > 0:
                        numerator = math.gamma((j) + r[0])
                        denominator = math.factorial((j)) * math.gamma(r[0])
                        gamma_term = numerator / denominator
                        pmf= (1 - phi[i]) * gamma_term * (1 + lambdaY[i] / r[0])**(-r[0]) * (1 + r[0] / lambdaY[i])**(-(j))
                cdf[i] += pmf
    return cdf

#### Tests

##### Definition der Frank-copula

In [25]:
# Frank Copula, theta fehlt
def Frank_copula(u_1, u_2, theta):
    numerator = (np.exp(-theta[0]*u_1) - 1) * (np.exp(-theta[0]*u_2) - 1)
    denominator = np.exp(-theta[0]) - 1
    fraction = numerator / denominator
    return (-1 / theta[0]) * np.log(1 + fraction)

##### Definition der Likelihoodfunktion

In [26]:
# alpha_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2 sind state dependent
# beta_o, r sind state independent
def likelihood_function(params):
    
    alpha_size = anzahl_states**2
    gamma_size = anzahl_states**2 * 4

    alpha = params[:alpha_size]
    gamma = params[alpha_size:alpha_size+gamma_size]
    alpha_o = params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
    beta_o = params[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
    delta_0 = params[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
    delta_1 = params[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
    alpha_p = params[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
    beta_p_1 = params[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
    beta_p_2 = params[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
    r = params[alpha_size+gamma_size+anzahl_states*6+1:alpha_size+gamma_size+anzahl_states*6+2]
    theta = params[alpha_size+gamma_size+anzahl_states*6+2:]

# Verwenden Sie die extrahierten Parameter in Ihrem Code
# ...


# Verwenden Sie die extrahierten Parameter in Ihrem Code
# ...

    #alpha, gamma, alpha_o, beta_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, r, theta = params
    ### Erstellung einer Liste H die alle H_it enthalten
    H = []
    
    ### Erstellung leerer 1xanzahl_states arrays für jeden Kunden und jeden Monat
    for iid in df_trans_werte["iid"]:
        for monat in range(1,16):
                empty_array = np.zeros((1, anzahl_states))
                H.append(empty_array)
                
                
    for monat in range(1,16):
        for state in range(1, anzahl_states +1):
            
            # Berechnung von arrays für die Verteilungsfunktionen
            F_O_state_monat_berechnung = F_O_state_monat(alpha_o, beta_o, state, monat)
            
            F_O_state_monat_berechnung_minusEins = F_O_state_monat_minusEins(alpha_o, beta_o, state, monat)
            
            F_Y_state_monat_berechnung = F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r)
            
            F_Y_state_monat_berechnung_minusEins = F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r)
            
            # Berechnung der gemeinsamen Verteilungen
            H_state_monat_berechnung =\
                            Frank_copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung, theta) -\
                            Frank_copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung, theta) -\
                            Frank_copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung_minusEins, theta) +\
                            Frank_copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung_minusEins, theta)
            
            # Speicherung der Werte in den leeren arrays
            for index in range(len(H_state_monat_berechnung)):
                H[index*len(range(1,16)) + (monat -1)][0][state-1] = H_state_monat_berechnung[index]
                #H[(monat-1)*len(df_trans_werte["iid"]) - 1 + index][0][state-1] = H_state_monat_berechnung[index]
                
    # Überführen der Arrays in Diagonalmatrizen
    H = [np.diag(arr[0][:]) for arr in H]

    # Startverteilung ist (1, 0,..., 0)
    startverteilung = np.zeros((1, anzahl_states))
    startverteilung[0, 0] = 1
    
    # Transitionmatrix Q, liste von 3x3 Arrays, wobei für Monate 2, ..., 15 die Q_it angegeben werden und die Q_it fü die
    # nächsten Kunden dahinter angehängt werden
    Q = probability_Q_state_monat(alpha, gamma)
    
    eins_vektor = np.ones((anzahl_states, 1))
    
    likelihood = np.array([])

    for j in range(len(df_trans_werte)):
        result = startverteilung
        result = np.dot(result, H[15 * j])
        H_iid = np.eye(anzahl_states)
        for monat in range(2,16):
            H_iid_monat = np.dot(H[j * 15 + (monat-1)], Q[j * 14 + (monat-2)])
            H_iid = np.dot(H_iid, H_iid_monat)
        result = np.dot(result, H_iid)
        result = np.dot(result, eins_vektor)
        likelihood = np.concatenate((likelihood, result[0]))
        
    return abs(np.sum(np.log(likelihood)))

#### Tests

# Berechnung der Likelihoodfunktion und Schätzung der Parameter

In [27]:
anzahl_states = 3

# Parameter für Transitionmatrix
#alpha = np.zeros((anzahl_states, anzahl_states)).flatten()
values_quelle_alpha = [0, -2.647, -1.432, 0, 2.829, -3.815, 0, -2.398, 0.61]
alpha = np.array(values_quelle_alpha)
gamma = np.ones((anzahl_states, anzahl_states, 4)).flatten()

# Parameter für CEOM
#alpha_o = np.zeros(anzahl_states)
values_quelle_alpha_o = [0.1, 0.1, 0.1]
alpha_o = np.array(values_quelle_alpha_o)
#beta_o = np.zeros(1)
values_quelle_beta_o = [0]
beta_o = np.array(values_quelle_beta_o)

# Parameter für CPM
delta_0 = np.ones(anzahl_states)
delta_1 = np.ones(anzahl_states)
alpha_p = np.ones(anzahl_states)
beta_p_1 = [0.1] * anzahl_states
beta_p_2 = [0.1] * anzahl_states
r = np.array([1.5])

# Parameter für Frank_copula
theta = np.array([0.324])


In [ ]:
# Setzen Sie die Startwerte
x0 = np.concatenate((
    alpha,
    gamma,
    alpha_o,
    beta_o,
    delta_0,
    delta_1,
    alpha_p,
    beta_p_1,
    beta_p_2,
    r,
    theta))

# Definiere bounds
bounds = [(None, None)] * (len(x0) - 2) + [(1.1, 1.2)] + [(0.1, None)]

# Rufen Sie minimize() auf und übergeben Sie die zusätzlichen Parameter mit args=
result = minimize(likelihood_function, x0, bounds = bounds)
                  #args=(alpha, gamma, alpha_o, beta_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, r, theta), bounds=bounds)

# Gib die geschätzten Parameter aus
print(result.x)

In [ ]:
# Berechne die Likelihood für die optimalen Parameter
likelihood = likelihood_function(result.x)
print("Likelihood für optimale Parameter:")
print(likelihood)

# Code Graveyard

Vorherige Definition der Verteilungsfunktion von O_it über Schleifen

##### Definition der Erfolgswahrscheinlichkeit p_it als 1 x anzhal_states array 
def p_alt(iid, t, alpha_o, beta_o):
    LO_iddt = df_LO.loc[df_LO['iid'] == iid, t].values[0]
    p = exp(alpha_o + beta_o * np.log(LO_iddt)) / (1 + exp(alpha + beta * log(LO_iddt)))
    return p

#Wahrscheinlichkeiten werden in Array von Arrays zurückgegeben
def F_O_iidt_alt(alpha_o, beta_o):
    # P_O_iidt_speicher als Zwischenspeicher
    P_O_iidt_speicher_gesamt = []
    
    for iid in trans["iid"]:
        # P_O_iidt_speicher als Zwischenspeicher für jede iid
        P_O_iidt_speicher_jede_iid = []
        
        for t in range(1, 16):
            # Defintion der Erfolgswahrscheinlichkeit p_it mit alpha_o_gegeben_s und beta_o als Paramater
            # erfolgs_p = p(iid, t, alpha_o, beta_o)
            
            # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
            EM_iidt = df_HY.loc[df_HY['iid'] == iid, t].values[0]
            
            # o_iidt: Anzahl geöffneter Mails von Kunde iid in Monat t
            o_iidt = df_NO.loc[df_NO['iid'] == iid, t].values[0]
            
            # Wahrscheinlichkeiten der Binomialverteilung für jeden Zustand
            P_O_it = binom.cdf(o_iidt, EM_iidt, p(iid, t, alpha_o, beta_o))
        
            # Speichern der Wahrscheinlichkeiten im Zwischenspeicher
            P_O_iidt_speicher_jede_iid = P_O_iidt_speicher_jede_iid.extend(P_O_it)

        # In einer neuen Zeile für jede iid in dem Zwischenspeicher abspeichern
        P_O_iidt_speicher_gesamt = np.vstack([P_O_iidt_speicher_gesamt, P_O_iidt_speicher])
        
    return P_O_iidt_speicher_gesamt


##### Tests mit Diagonalmatrizen
            # Erstellen des Namens der Diagonalmatrix
            # name = f"diagonal_matrix_PO_{iid}_{t}"
        
            # Erstellen der Diagonalmatrix mit den Wahrscheinlichkeiten und dem Namen
            # diagonal_matrix = np.diag(probabilities)
            # diagonal_matrix.name = name

# Code wo veruscht wurde die arrays mit namen zu identifizieren

#### Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state_t, monat):
    
    state_t_minusEins = state_t_minusEins -1
    state_t = state_t -1
    str_monat = str(monat)
    
    # Definition der X_t_minusEins
    # verwendete Parameter
    EM_monat_minusEins = df_HY[str_monat]
    Y_monat_minusEins = df_trans_werte[str_monat]
    O_monat_minusEins = df_NO[str_monat]
    # Definition
    X_t_minusEins = [int(O_monat_minusEins > 0), int(Y_monat_minusEins > 0),\
                     EM_monat_minusEins, np.square(EM_monat_minusEins)]
    
    array_v = alpha[12*state_t_minusEins + 4*state + state] + gamma[12*state_t_minusEins + 4*state + state] * X_t_minusEins

    return array_v

##### Definition der Transitionmatrix
def probability_Q_state_monat(alpha, gamma):
    
    ### Erstellung leerer anzahl_states x anzahl_states arrays für jeden Kunden und jeden Monat
    Q = []
    for iid in df_trans_werte["iid"]:
        for monat in range(2,16):
                empty_array = np.zeros((anzahl_states, anzahl_states))
                Q.append(empty_array)
                
                #name = f"matrix_Q_{iid}_{monat}"
                #empty_array.name = name
                
    for monat in range(2,16):
        
        for state_t_minusEins in range(1, anzahl_states +1):
            
            # Erste Spalte besteht aus Nullen, da eine Trasition zurück in state 1 nicht möglich ist
            for state in range(2, anzahl_states +1):
                
                q_array_iid_monat_state = Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state_t, monat)
                #name = f"Q_{iid}_{monat}"

                # Überführen zu df und Hinzufügen einer iid Spalte zur Identifikation
                q_array_iid_monat_state = pd.DataFrame(q_array_iid_monat_state)
                q_array_iid_monat_state["iid"] = df_trans_werte["iid"]

                # Speicherung der Werte in den leeren arrays
                for index in q_array_iid_monat_state:
                    #name = f"matrix_Q_{iid}_{monat}"
                    #array_iid_monat = Q[name]
                    
                    #Neue Definition ohne die Verwedung von Namen
                    Q[index][state_t_minusEins-1, state-1] =\
                                    q_array_iid_monat_state[index]
                   
                    #array_iid_monat[state_t_minusEins-1, state-1] =\
                                    #q_array_iid_monat_state[q_array_iid_monat_state["iid"] == "iid"]
    
    return Q

#### alpha_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2 sind state dependent
#### beta_o, r sind state independent
def likelihood_function(alpha, gamma, alpha_o, beta_o, delta_0, delta_1,\
                        alpha_p, beta_p_1, beta_p_2, r, theta):
    
    ### Erstellung einer Liste H die alle H_it enthalten
    H = []
    
    ### Erstellung leerer 1xanzahl_states arrays für jeden Kunden und jeden Monat
    for iid in df_trans_werte["iid"]:
        for monat in range(1,16):
                empty_array = np.zeros((1, anzahl_states))
                H.append(empty_array)
                
                name = f"diagonal_matrix_H_{iid}_{monat}"
                empty_array.name = name
                
    for monat in range(1,16):
        for state in range(1, anzahl_states +1):
            
            # Berechnung von arrays für die Verteilungsfunktionen
            F_O_state_monat_berechnung = F_O_state_monat(alpha_o, beta_o, state, monat)
            
            F_O_state_monat_berechnung_minusEins = F_O_state_monat_minusEins(alpha_o, beta_o, state, monat)
            
            F_Y_state_monat_berechnung =\
                            F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r)
            
            F_Y_state_monat_berechnung_minusEins =\
                            F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r)
            
            # Berechnung der gemeinsamen Verteilungen
            H_state_monat_berechnung =\
                            Frank_Copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung, theta) -\
                            Frank_Copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung, theta) -\
                            Frank_Copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung_minusEins, theta) +\
                            Frank_Copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung_minusEins, theta)
            
            # Überführen zu df und Hinzufügen einer iid Spalte zur Identifikation
            H_state_monat_berechnung = pd.DataFrame(H_state_monat_berechnung)
            H_state_monat_berechnung["iid"] = df_trans_werte["iid"]
            
            # Speicherung der Werte in den leeren arrays
            for iid in H_state_monat_berechnung["iid"]:
                name = f"diagonal_matrix_H_{iid}_{monat}"
                H[name][state-1] = H_state_monat_berechnung[H_state_monat_berechnung["iid"] == "iid"]
                
    # Überführen der Arrays in H in Diagonalmatrizen
    for array in H:
        H[array] = np.diag(H[array])
        
    # Berechnung der Likelihood Function
    likelihood = 1
    
    # Startverteilung ist (1, 0,..., 0)
    startverteilung = np.zeros((1, anzahl_states))
    startverteilung[0, 0] = 1
    
    # Transitionmatrix Q
    Q = probability_Q_state_monat(alpha, gamma)
    
    Q_iid_prod = []
    for iid in df_trans_werte["iid"]:
        Q_iid = np.dot(startverteilung, H[f"diagonal_matrix_H_{iid}_1"])

        for monat in range(2,16):
            name = f"diagonal_matrix_H_{iid}_{monat}"
            name_Q = f"matrix_Q_{iid}_{monat}"
            H_iid_monat = np.dot(H[name], Q[name_Q])
            H_iid = np.dot(H_iid, H_iid_monat)
            
        Q_iid_prod.append(np.dot(Q_iid, H_iid))
        
    return np.prod(Q_iid_prod)

# ZINBD Verteilungsfunktion mit Model modelliert

#### Definition der Wahrscheinlichkeit des Eintretens eines Erfolges im Einzelversuch phi als anzahl_iids x T df
def phi_state_monat(delta_0, delta_1, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    LY_monat = df_LY[str_monat]
    array_phi = 1 / (1 + np.exp(delta_0[state] + delta_1[state] * np.log(LY_monat)))
    
    return array_phi

#### Definition der lambdaY als anzahl_iids x 1 array für monat und state
def lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    EM_monat = df_HY[str_monat]
    
    array_lambdaY = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
    
    return array_lambdaY

#### Array der Verteilungsfunktion der Käufe Y für Monat und state
def F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # Erstellen Sie ein Zero-Inflated Negative Binomial Modell
    zinb_model = ZeroInflatedNegativeBinomialP(endog=None, exog=None, exposure=None, offset=None, missing='none')

    # Setzen Sie die offenen Parameter als Modellparameter
    zinb_model.update({'params': [1- phi_state_monat(delta_0, delta_1, state, monat),\
                                  lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat),\
                                  r]})
    
    # y_iid ist das DataFrame mit der Anzahl an Käufen
    y_monat = df_trans_werte[str_monat]
    
    # Verteilungsfunktion der ZINBD für jeden Zustand state
    F_Y_it = zinb_model.cdf(y_monat)
       
    return F_Y_it

#### Array der Verteilungsfunktion der Käufe Y-1 für Monat und state
def F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat, r):
    
    # Berechnung der Werte minus Eins für spätere Berechnung der gemeinsamen Verteilung via Frank-Copula
    df_trans_werte_minusEins = df_trans_werte.loc[:, df_NO_werte.columns != 'iid'] -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # Erstellen Sie ein Zero-Inflated Negative Binomial Modell
    zinb_model = ZeroInflatedNegativeBinomialP(endog=None, exog=None, exposure=None, offset=None, missing='none')

    # Setzen Sie die offenen Parameter als Modellparameter
    zinb_model.update({'params': [1- phi_state_monat(delta_0, delta_1, state, monat),\
                                  lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat),\
                                  r]})
    
    # y_iid ist das DataFrame mit der Anzahl an Käufen
    y_monat = df_trans_werte_minusEins[str_monat]
    
    # Verteilungsfunktion der ZINBD für jeden Zustand state
    F_Y_it = zinb_model.cdf(y_monat)
       
    return F_Y_it

# CEOM als BD

#### Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def probability_O_state_monat(alpha_o, beta_o, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # Erhalten der Werte für die Verarbeitung in der Formel
    LO_monat = df_LO[str_monat]
    array_p = np.exp(alpha_o[state] + beta_o[0] * np.log(LO_monat)) / (1 + np.exp(alpha_o[state] + beta_o[0] * np.log(LO_monat)))

    return array_p

#### Array der Verteilungsfunktion der Öffungen O für Monat und state
def F_O_state_monat(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_werte[str_monat]
    
    # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
    EM_monat = df_HY[str_monat]
    
    probs = probability_O_state_monat(alpha_o, beta_o, state, monat)
    array_F_O_berechnung = binom.cdf(o_monat, EM_monat, probs)
         
    return array_F_O_berechnung

#### Array der Verteilungsfunktion der Öffungen O -1 für Monat und state
def F_O_state_monat_minusEins(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_minusEins[str_monat]
    
    # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
    EM_monat = df_HY[str_monat]
    
    probs = probability_O_state_monat(alpha_o, beta_o, state, monat)
    array_F_O_berechnung = binom.cdf(o_monat, EM_monat, probs)

    return array_F_O_berechnung

#### Umständliche Defiitio Matrixmultiplikation
    Q_iid_prod = np.array([1])
    for j in range(len(df_trans_werte)):
        Q_iid = np.dot(startverteilung, H[15 * j])
        H_iid = np.eye(anzahl_states)
        for monat in range(2,16):
            H_iid_monat = np.dot(H[j * 15 + (monat-1)], Q[j * 14 + (monat-2)])
            H_iid = np.dot(H_iid, H_iid_monat)
            
        Q_iid_prod.append(np.dot(Q_iid, H_iid))